# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup


In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.


In [1]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría ordenar una licuadora.


In [2]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This language is French.


In [3]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)


French: ```Je veux commander un ballon de basket```
Spanish: ```Quiero ordenar una pelota de baloncesto```
English: ```I want to order a basketball```



In [4]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)


Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?



### Universal Translator

Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!


In [5]:
user_messages = [
    "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal
    "Mi monitor tiene píxeles que no se iluminan.",  # My monitor has pixels that are not lighting
    "Il mio mouse non funziona",  # My mouse is not working
    "Mój klawisz Ctrl jest zepsuty",  # My keyboard has a broken control key
    "我的屏幕在闪烁",  # My screen is flashing
]

In [6]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")


Original message (The language is French.): La performance du système est plus lente que d'habitude.
The performance of the system is slower than usual.

시스템의 성능이 평소보다 느립니다. 

Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 밝아지지 않는 픽셀이 있습니다." 

Original message (The language is Italian.): Il mio mouse non funziona
English: "My mouse is not working."
Korean: "내 마우스가 작동하지 않습니다." 

Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
English: "My Ctrl key is broken"
Korean: "내 Ctrl 키가 고장 났어요" 

Original message (The language is Chinese.): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜박거립니다.       



## Tone Transformation

Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [7]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)


Dear Sir/Madam,

I hope this letter finds you well. My name is Joe, and I am writing to bring your attention to a specification document regarding a standing lamp. 

I kindly request that you take a moment to review the attached document, as it provides detailed information about the features and qualities of the aforementioned standing lamp. 

Thank you for your time and consideration. I look forward to discussing this matter further with you.

Yours sincerely,
Joe



## Format Conversion

ChatGPT can translate between formats. The prompt should describe the input and output formats.


In [8]:
data_json = {
    "resturant employees": [
        {"name": "Shyam", "email": "shyamjaiswal@gmail.com"},
        {"name": "Bob", "email": "bob32@gmail.com"},
        {"name": "Jai", "email": "jai87@gmail.com"},
    ]
}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)


<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>

</body>
</html>



In [9]:
from IPython.display import display, Markdown, Latex, HTML, JSON

display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response.

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.


In [10]:
text = [
    "The girl with the black and white puppies have a ball.",  # The girl has a ball.
    "Yolanda has her notebook.",  # ok
    "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
    "Your going to need you’re notebook.",  # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?",  # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty",  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)


The girl with the black and white puppies has a ball.
No errors found.
It's going to be a long day. Does the car need its oil changed?
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check chatGPT for spelling ability.
    


In [12]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)


Got this for my daughter for her birthday because she keeps taking mine from my room. 
Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. 
However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. 
Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. 
On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.



In [14]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))


Title: A Delightful Gift for All Ages: A Review of the Adorable Panda Plush Toy

Introduction

Purchasing a gift that appeals to both children and adults can be quite a challenge. However, I recently stumbled upon a remarkable solution - the Panda Plush Toy. In this review, I will share my experience with this delightful gift, highlighting its irresistible charm, exceptional softness, and unexpected versatility. While there were a few minor imperfections, the overall quality and prompt delivery exceeded my expectations. Join me as I delve into the enchanting world of this captivating panda companion.

Review

As a parent, I understand the struggle of keeping personal belongings safe from curious little hands. That's why I decided to surprise my daughter with the Panda Plush Toy for her birthday. To my surprise, this adorable companion not only captivated her heart but also managed to capture mine. Its irresistible charm transcends age boundaries, making it a perfect gift for both children and adults alike.

The first thing that struck me about this plush toy was its exceptional softness. Crafted with meticulous attention to detail, the plush material envelops you in a cloud of comfort. Its velvety texture is a testament to the superior craftsmanship that went into its creation. I couldn't resist running my fingers through its fur, and I must admit, I found myself snuggling with it more often than I anticipated.

While the Panda Plush Toy is undeniably endearing, I did notice a minor flaw in its design. One of the ears appeared slightly lower than the other, which seemed unintentional. However, this asymmetry did not detract from its overall appeal. In fact, it added a touch of uniqueness, making each panda toy one-of-a-kind.

In terms of size, I must admit that I expected a slightly larger plush toy for the price I paid. Although it is compact, its portability is an advantage, allowing my daughter to take it everywhere she goes. However, for those seeking a larger option, it may be worth exploring alternative choices within the same price range.

Lastly, I was pleasantly surprised by the prompt delivery of the Panda Plush Toy. It arrived a day earlier than expected, providing me with an opportunity to experience its charm firsthand. This unexpected bonus allowed me to appreciate the toy's exceptional quality and ensured that it was in perfect condition before presenting it to my daughter.

Conclusion

In conclusion, the Panda Plush Toy has proven to be an enchanting gift that transcends age boundaries. Its irresistible charm, exceptional softness, and unexpected versatility make it a delightful companion for both children and adults. While a minor design flaw and its compact size may be worth considering, the overall quality and prompt delivery exceeded my expectations. If you are seeking a gift that will bring joy and comfort to your loved ones, the Panda Plush Toy is an excellent choice.
